In [6]:
# Importing the needed libraries

import io
import requests
import numpy as np
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import t

import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.initializers import HeNormal
from keras.wrappers.scikit_learn import KerasRegressor

import warnings
warnings.filterwarnings("ignore")

#######################################################################################################################################

# Load data 
url = 'https://drive.switch.ch/index.php/s/TeDwnbYsBKRuJjv/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))

#######################################################################################################################################

# T1 Linear Regression Model

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# creating arrays for the matrix X using the data x
ones = np.ones(x.shape[0])     # creating an array of ones with the same length as x
x1 = x[:,0]                    # selecting the first column of x: x1
x2 = x[:,1]                    # selecting the second column of x: x2
sin_x2 = np.sin(x2)            # computing the sine of each element of x2
x1_x2 = x1*x2                  # computing the product of x1 and x2

# creating the matrix X for linear regression by horizontally stacking the arrays
X = np.hstack((ones.reshape(-1, 1), x1.reshape(-1, 1), x2.reshape(-1, 1), sin_x2.reshape(-1, 1), x1_x2.reshape(-1, 1)))

# splitting the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=80)

# creating a linear regression model 
lr = LinearRegression(fit_intercept=False)

# training process: fit the model to the training data 
lr.fit(X_train, y_train)

# predictions using the lr model
y_pred = lr.predict(X_test)

# mse of the model on the test set
mse = mean_squared_error(y_test,y_pred)

# coefficient of the linear regression model
theta = lr.coef_

# rounding the coefficients
for index in range(len(theta)):
    theta[index] = round(theta[index],4)

# KFold approach to calculate the expected performance at task of the Linear Regression model and its confidence level

# Create a KFold object
kfold = KFold(n_splits=10, shuffle=True, random_state=80)

# Perform k-fold cross-validation
scores = cross_val_score(lr, X, y, scoring='neg_mean_squared_error', cv=kfold)

# The cross_val_score function by default calculates negative MSE for scoring

mse_scores_lin = - scores

# 95% confidence interval 

# confidence level
confidence = 0.95

# number of scores (MSEs - 10 in this case)
n = len(mse_scores_lin)

# standard deviation of the scores (MSEs)
std_err = np.std(mse_scores_lin) / np.sqrt(n)

# margin error
margin_error95_lin = std_err * t.ppf((1 + confidence) / 2, n - 1)

# print settings
print("-" * 130) 
print("T1 - LINEAR REGRESSION")
print(f"theta: {theta}")
print(f"Mean Squared Error on the test set (Linear Regression): {mse:.5f}")
print("Performance at task - kfold Cross Validation (k=10) - Estimated MSE for this model:  {:.5f} +- {:.5f}".format(np.mean(mse_scores_lin),margin_error95_lin))

# save the T1 - Linear Regression model to a pickle file
with open('LinearRegressionModel.pickle', 'wb') as f:
    pickle.dump(lr, f)

print("-" * 130)  

#######################################################################################################################################

# T2 Random Forest Regression Model

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# Split the data into 70% training set, 30% test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=80)

# initialization the param_grid for the GridSearchCV when fitting the training set
param_grid = {'n_estimators': [100,300,400],'max_depth': [30,40,50],'random_state':[80]}

# initialization of the model
model_type = RandomForestRegressor()

# initialization of the grid_search
# specify scoring as 'neg_mean_squared_error'
grid_search = GridSearchCV(model_type, param_grid, cv=3, scoring='neg_mean_squared_error')

# fitting the grid_search
grid_search.fit(X_train, y_train)

# this is the best model, based on the MSE of the validation set using a cv of 3
rf = grid_search.best_estimator_

# predict using best model
y_test_pred = rf.predict(X_test)

# mse of the model on the test set
test_mse = mean_squared_error(y_test, y_test_pred)

# KFold approach to calculate the expected performance at task of the Random Forest Regression model and its confidence level

kfold = KFold(n_splits=10, shuffle=True, random_state=80)

# performing k-fold cross-validation
scores = cross_val_score(rf, x, y, scoring='neg_mean_squared_error', cv=kfold)

# the cross_val_score function by default calculates negative MSE for scoring

mse_scores_rf = - scores

# 95% confidence interval 

# confidence level
confidence = 0.95

# number of scores (MSEs - 10 in this case)
n = len(mse_scores_rf)

# standard deviation of the scores (MSEs)
std_err = np.std(mse_scores_rf) / np.sqrt(n)

# margin error
margin_error95_rf = std_err * t.ppf((1 + confidence) / 2, n - 1)

# print settings

# getting all the parameters of the best model
params = rf.get_params()
# selecting the tuned ones (here random_state is fixed to 80 but was included for semplicity of the script)
parameters_of_interest = {key: params[key] for key in ['max_depth', 'n_estimators', 'random_state']}
# storing the tuned parameters in the params_str variable, for printing reasons
params_str = ', '.join(f'{k}={v}' for k, v in parameters_of_interest.items())

print(f"T2 - RANDOM FOREST REGRESSOR - {params_str}")
print(f"Mean Squared Error on the test set (Random Forest Regressor): {test_mse:.5f}")
print("Performance at task - kfold Cross Validation (k=10) - Estimated MSE for this model:  {:.5f} +- {:.5f}".format(np.mean(mse_scores_rf),margin_error95_rf))

# save the T2 - Random Forest Regressor model to a pickle file
with open('RandomForestRegressorModel.pickle', 'wb') as f:
    pickle.dump(rf, f)

print("-" * 130) 

#######################################################################################################################################

# T1 vs T2 - t test

# T1

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# X matrix and splitting procedure
ones = np.ones(x.shape[0]) ; x1 = x[:,0]  ; x2 = x[:,1]  ; sin_x2 = np.sin(x2)  ; x1_x2 = x1*x2                  
X = np.hstack((ones.reshape(-1, 1), x1.reshape(-1, 1), x2.reshape(-1, 1), sin_x2.reshape(-1, 1), x1_x2.reshape(-1, 1)))

# spliting the data into 70% training set, 30% test set (same random_state as above)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=80)

# predictions using the lr model
y_pred_lr = lr.predict(X_test)

# mse of the lr model on the test set
mse_lr = mean_squared_error(y_test,y_pred_lr)

# residuals of the lr model
res_lr = y_test - y_pred_lr

# T2

# spliting the data into 70% training set, 30% test set (same random_state as above)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=80)

# predictions using the rf model
y_pred_rf = rf.predict(X_test)

# mse of the rf model on the test set
mse_rf = mean_squared_error(y_test,y_pred_rf)

# residuals of the rf model
res_rf = y_test - y_pred_rf

# -- T Test ----------------------------------------------------------------------------------------------------

# MSEs
mse1 = mse_lr ; mse2 = mse_rf 

# Ls
l1 = len(y_test) ; l2 = len(y_test)

# S1 and S2
s1 = (1/(l1-1)) * np.sum ((((res_lr)**2) - mse_lr )**2)
s2 = (1/(l2-1)) * np.sum ((((res_rf)**2) - mse_rf )**2)

# T test value
T = (mse1 - mse2) / (np.sqrt ((s1/l1)+(s2/l2)))

# print setting
print("T1 vs T2 - LINEAR REGRESSION VS RANDOM FOREST REGRESSION")
print(f"T-test value: {T:.3f}")

if abs(T)>1.96:
    print(f"We can conclude that T2 performed statistically better than T1 with a 95% confidence interval since abs({T:.3f}) > 1.96")
else: 
    print(f"We cannot conclude that T2 performed statistically better than T1 with a 95% confidence interval since abs({T:.3f}) < 1.96")

print("-" * 130) 

#######################################################################################################################################

# T3 Feed Feedforward Neural Network Model - Bonus Point

# Setting random seeds for reproducibility of results
keras.utils.set_random_seed(80)

# x is a Numpy array with the inputs
x = data.f.x
# y is a Numpy array with the targets
y = data.f.y


#  Model ---------------------------------------------------------------------------------------------------------------
# initializing the model
nn = Sequential()  

# Split the data into 70% training set, 15% validation set, and 15% test set
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=80)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=80)

# 1: dense layer with 64 neurons, using 'tanh' activation and input dimensions from X_train, he_init kernel_initializer
# The He Normal initialization function initializes the weights of the layers using a normal distribution
# with a mean of zero and a standard deviation calculated based on the number of input units to the layer

he_init = HeNormal(seed=80)

nn.add(Dense(64, input_dim=X_train.shape[1], activation='tanh', kernel_initializer=he_init))

# 2: dense layer with 32 neurons and 'tanh' activation, he_init kernel_initializer
nn.add(Dense(32, activation='tanh', kernel_initializer=he_init))

# 3: final dense layer with 1 neuron (output layer) 
nn.add(Dense(1))

# ------------------------------------------------------------------------------------------------------------------------

# compiling the model with the mean_squared_error loss function and the adam optimizer
nn.compile(loss='mean_squared_error', optimizer='adam')

# defining an early stopping function - to prevent overfitting
es = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)

# fitting the model with its 'history' (to retrieve the final number of epochs in the next line)
history = nn.fit(X_train, y_train, epochs=400, batch_size=32, verbose=0, 
                  validation_data=(X_val, y_val), callbacks=[es])

# retrieving the number of epochs that balances the bias-variance trade-off.
num_epochs_trained = len(history.history['loss'])

# how many epochs?? - printing this info in the console
if num_epochs_trained == 400:
    print('The Dense Neural Network trained for 400 epochs - The early stopping function was not triggered')
else: 
    print(f'The Dense Neural Network trained for {num_epochs_trained} epochs - The early stopping function was triggered')

# mse of the Feed Feedforward Neural Network model on the test set 
test_mse = nn.evaluate(X_test, y_test, verbose=0)

# print settings
print(f"T3 - DENSE NEURAL NETWORK: 64 (tanh) - 32 (tanh) - 1, {num_epochs_trained} epochs - batch size = 32 - adam optimizer")
print(f"Mean Squared Error on the test set (Neural Network): {test_mse:.5f}")

# KFold approach to calculate the expected performance at task of the Feed Feedforward Neural Network model and its confidence level

input_dim = x.shape[1]

# defining a function that returns a compiled model - Keras models by default are not compatible with the scikit_learn kfold framework
def create_model(input_dim):
    nn = Sequential() ; he_init = HeNormal(seed=80)
    nn.add(Dense(64, input_dim=input_dim, activation='tanh', kernel_initializer=he_init))
    nn.add(Dense(32, activation='tanh', kernel_initializer=he_init))
    nn.add(Dense(1)) ; nn.compile(loss='mean_squared_error', optimizer='adam')
    return nn

# wrapping the Keras model in an estimator compatible with scikit_learn - epochs=num_epochs_trained!
nn_sklearn = KerasRegressor(build_fn=lambda: create_model(input_dim), epochs=num_epochs_trained, batch_size=32, verbose=0)

# using the Keras model with the cross_val_score tool in scikit_learn  
kfold = KFold(n_splits=10, shuffle=True, random_state=80)
scores = cross_val_score(nn_sklearn, x, y, scoring='neg_mean_squared_error', cv=kfold)

# the cross_val_score function by default calculates negative MSE for scoring

mse_scores_nn = - scores

# 95% confidence interval 

# confidence level
confidence = 0.95

# number of scores (MSEs - 10 in this case)
n = len(mse_scores_nn)

# standard deviation of the scores (MSEs)
std_err = np.std(mse_scores_nn) / np.sqrt(n)

# margin error
margin_error95_nn = std_err * t.ppf((1 + confidence) / 2, n - 1)

# print settings

print("Performance at task - kfold Cross Validation (k=10) - Estimated MSE for this model:  {:.5f} +- {:.5f}".format(np.mean(mse_scores_nn), margin_error95_nn))

print("-" * 130) 

# save the T3 - Neural Network model to a pickle file
with open('NeuralNetworkModel.pickle', 'wb') as f:
    pickle.dump(nn, f)

#######################################################################################################################################

----------------------------------------------------------------------------------------------------------------------------------
T1 - LINEAR REGRESSION
theta: [ 1.2943 -0.0585 -0.5894  0.4879  0.048 ]
Mean Squared Error on the test set (Linear Regression): 0.68263
Performance at task - kfold Cross Validation (k=10) - Estimated MSE for this model:  0.71681 +- 0.05555
----------------------------------------------------------------------------------------------------------------------------------
T2 - RANDOM FOREST REGRESSOR - max_depth=30, n_estimators=400, random_state=80
Mean Squared Error on the test set (Random Forest Regressor): 0.02161
Performance at task - kfold Cross Validation (k=10) - Estimated MSE for this model:  0.01943 +- 0.00115
----------------------------------------------------------------------------------------------------------------------------------
T1 vs T2 - LINEAR REGRESSION VS RANDOM FOREST REGRESSION
T-test value: 13.841
We can conclude that T2 performed st

In [8]:
# Import libraries

import requests
import io
import joblib
import numpy as np
import warnings

warnings.filterwarnings("ignore")

#####################################################################################

def evaluate_predictions(y_true, y_pred):
    """
    Evaluates the mean squared error between the values in y_true and the values
    in y_pred.
    ### YOU CAN NOT EDIT THIS FUNCTION ###
    :param y_true: Numpy array, the true target values from the test set;
    :param y_pred: Numpy array, the values predicted by your model.
    :return: float, the mean squared error between the two arrays.
    """
    assert y_true.shape == y_pred.shape
    return ((y_true - y_pred) ** 2).mean()

def load_model(filename):
    """
    Loads a Scikit-learn model saved with joblib.dump.
    This is just an example, you can write your own function to load the model.
    Some examples can be found in src/utils.py.
    :param filename: string, path to the file storing the model.
    :return: the model.
    """
    model = joblib.load(filename)

    return model

#########################################################################################################
######################################################################################################### 

# Load the data
# This will be replaced with our private test data when grading the assignment

# Load data from url
url = 'https://drive.switch.ch/index.php/s/TeDwnbYsBKRuJjv/download'
response = requests.get(url)
data = np.load(io.BytesIO(response.content))

# x is a Numpy array of shape (n_samples, n_features) with the inputs
x = data.f.x
# y is a Numpy array of shape (n_samples, ) with the targets
y = data.f.y

# Data Preprocessing ---------> y in this case needed to be reshaped. If not... please remove this part
y = y.reshape(-1,1)

#########################################################################################################
#########################################################################################################

# Load the trained model
nn_model = load_model("./NeuralNetworkModel.pickle")

# Predict on the given samples
y_pred = nn_model.predict(x,verbose=0)
print("-" * 130)

############################################################################
# STOP EDITABLE SECTION: do not modify anything below this point.
############################################################################

# Evaluate the prediction using MSE
mse = evaluate_predictions(y_pred, y)
print(f'MSE on whole dataset: {mse}')

# NOTE: NOW THIS CELL IS NOT WORKING SINCE YOU NEED TO CHANGE THE INPUT.
# DO IT AND EVERYTHING RUNS SMOOTH

############################################################################

print("-" * 130)

----------------------------------------------------------------------------------------------------------------------------------
MSE on whole dataset: 0.012276736278965278
----------------------------------------------------------------------------------------------------------------------------------
